In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import pandas as pd
import pprint as pp
import os
import re
import pprint as pp
import tqdm
import pickle
import utils
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

Read the CSV

In [2]:
df2 = pd.read_csv('h1b_raw_data_2019.csv')

/home/jingbo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,14,15,21,23,27,28,29,30,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,49,58,59,60,62,65,67,68,69,70,71,72,73,74,77,79,80,81,83,86,88,89,90,91,92,93,94,95,98,99,100,101,102,104,107,109,110,111,112,113,114,115,116,122,123,125,128,130,131,132,133,134,135,136,137,143,144,146,149,151,152,153,154,155,156,157,158,161,163,164,165,167,170,172,173,174,175,176,177,178,179,185,186,188,191,193,194,195,196,197,198,199,200,206,207,209,212,214,215,216,217,218,219,220,221,224,226,227,228,230,233,235,236,237,238,239,240,241,245,247,248,249,251,254,258,259,260) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Clean up the data using a loop. Be patient, this takes 4 minuts on an AMD 3900X

In [4]:
clean_data = []
soc_code_lookup = {}
for i, row in tqdm.tqdm(df2.iterrows(), total=len(df2), miniters=int(50000)):
    
    data = {
        'VISA_CLASS': str((row['VISA_CLASS'])),
        'CASE_STATUS': str((row['CASE_STATUS'])),
        'SOC_TITLE': utils.get_soc_title_lookup(str((row['SOC_TITLE']))),
        'SOC_CODE': utils.cleanup_soc_code(str((row['SOC_CODE']))),
        'NAICS_CODE': str((row['NAICS_CODE'])),
        'EMPLOYER_NAME': utils.company_name_replace(str((row['EMPLOYER_NAME'])), 2),
        'AGENT_ATTORNEY_LAW_FIRM': utils.company_name_replace(str((row['AGENT_ATTORNEY_LAW_FIRM_BUSINESS_NAME'])), 2),
        'WORKSITE_ADDRESS1': str((row['WORKSITE_ADDRESS1_1'])),
        'WORKSITE_ADDRESS2': str((row['WORKSITE_ADDRESS2_1'])),
        'WORKSITE_CITY': str((row['WORKSITE_CITY_1'])),
        'WORKSITE_COUNTY': str((row['WORKSITE_COUNTY_1'])),
        'WORKSITE_STATE': utils.get_state_abbrev(str((row['WORKSITE_STATE_1']))),
        'WORKSITE_POSTAL_CODE': utils.cleanup_zip(str((row['WORKSITE_POSTAL_CODE_1']))),
        'WAGE_RATE_OF_PAY_FROM': float(row['WAGE_RATE_OF_PAY_FROM_1']),
        'WAGE_RATE_OF_PAY_TO': float(row['WAGE_RATE_OF_PAY_TO_1']),
        'WAGE_UNIT_OF_PAY': str((row['WAGE_UNIT_OF_PAY_1'])),
        'PREVAILING_WAGE': float(row['PREVAILING_WAGE_1']),
        'PW_UNIT_OF_PAY': str((row['PW_UNIT_OF_PAY_1'])),
        'PW_TRACKING_NUMBER': str((row['PW_TRACKING_NUMBER_1'])),
        'PW_WAGE_LEVEL': str((row['PW_WAGE_LEVEL_1'])),
        'PW_OES_YEAR': str((row['PW_OES_YEAR_1'])),
        'PW_OTHER_SOURCE': str((row['PW_OTHER_SOURCE_1'])),
        'PW_NON-OES_YEAR': str((row['PW_NON-OES_YEAR_1'])),
    }

    if data['SOC_CODE'] not in soc_code_lookup:
        soc_code_lookup[data['SOC_CODE']] = set()
    soc_code_lookup[data['SOC_CODE']].add(data['SOC_TITLE'])

    data['WAGE_RATE_ANNUAL'] = utils.get_okay_wage(data['WAGE_RATE_OF_PAY_FROM'], data['WAGE_UNIT_OF_PAY'])
    data['PREVAILING_WAGE_RATE_ANNUAL'] = utils.get_okay_wage(data['PREVAILING_WAGE'], data['PW_UNIT_OF_PAY'])
    
    data['WORKSITE_ADDRESS_CLEAN'] = utils.addr_abbrev_replace(utils.get_clean_street_address(data['WORKSITE_ADDRESS1'], data['WORKSITE_ADDRESS2']))
    data['WORKSITE_CITY_CLEAN'] = utils.city_regex(data['WORKSITE_CITY'])
    clean_data.append(data)
    #if i > 50 * 1000:
    #    break
    
clean_df = pd.DataFrame(clean_data)

100%|██████████| 664616/664616 [04:46<00:00, 2317.81it/s]


Save SOC code to SOC title lookup

In [ ]:
pickle.dump(soc_code_lookup, open('soc_code_lookup.pickle', 'wb'))

We want to lookup addresses, so we first get unique addresses, then query OpenStreetMap API for the GPS location.

In [5]:
unique_full_addr = list(zip(clean_df['WORKSITE_ADDRESS_CLEAN'], clean_df['WORKSITE_CITY_CLEAN'], clean_df['WORKSITE_POSTAL_CODE']))
unique_addrs, unique_addr_counts = np.unique(unique_full_addr, return_counts=True, axis=0)
sorted_unique_addr_count_args = np.argsort(unique_addr_counts)[::-1]

Map API has query limit. XD. You should not query all 100k or so unique addresses all together. You can only do around 1500~ at a time before being kicked out.

In [ ]:
address_lookup = {}

In [ ]:
for i in tqdm.tqdm(range(0, 1500), miniters=int(60)):
    orig_index = sorted_unique_addr_count_args[i]
    # print(f'{unique_addrs[orig_index]}: {unique_addr_counts[orig_index]}')
    
    search_string = ' '.join(unique_addrs[orig_index])
    gps = get_gps(search_string)
    # print(gps)
    
    address_lookup[tuple(unique_addrs[orig_index])] = gps

Then assign GPS addresses to the cleaned dataframe

In [ ]:
useful_df = clean_df[['VISA_CLASS', 'CASE_STATUS', 
                      'SOC_TITLE', 'SOC_CODE', 'NAICS_CODE',
                      'EMPLOYER_NAME', 'AGENT_ATTORNEY_LAW_FIRM',
                      'WORKSITE_ADDRESS_CLEAN', 'WORKSITE_CITY_CLEAN', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE',
                      'WAGE_RATE_ANNUAL', 'PREVAILING_WAGE_RATE_ANNUAL', 'PW_WAGE_LEVEL', 'PW_OES_YEAR',
                     ]]
useful_df = useful_df[(useful_df['CASE_STATUS'] == 'CERTIFIED') & (useful_df['VISA_CLASS'] == 'H-1B') ]
print(len(useful_df))

In [ ]:
def find_gps(row):
    # print(row)
    gps_key = tuple(row[['WORKSITE_ADDRESS_CLEAN', 'WORKSITE_CITY_CLEAN', 'WORKSITE_POSTAL_CODE']])
    if gps_key in address_lookup and address_lookup[gps_key] is not None:
        # print('--')
        gps = address_lookup[gps_key]
        # row['LATITUDE'] = gps[0]
        # row['LONGITUDE'] = gps[1]
        return gps

In [ ]:
tqdm.tqdm.pandas()

useful_df['GPS'] = useful_df.progress_apply(lambda row: find_gps(row), axis=1)
useful_df.head(100)

In [ ]:
pickle.dump(useful_df, open('useful_df.pickle', 'wb'))